### 引入库

In [ ]:
import datetime
import pandas as pd
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

### 策略类

In [ ]:
1.自定义的策略类继承了Backtrader的策略类(bt.Strategy)

2.类中使用params来保存参数，类中调用时可以以方法形式：self.params.period_sma5
params = (
        ('period_sma10', 10),
        ('period_sma30', 30)
    )

3.实例化期间，执行__init__()方法，此时会创建变量并赋值（order、buyprice…）和计算指标（sma10、sma30）。
def __init__(self):
    # 用于保存订单
    self.order = None
    # 订单价格
    self.buyprice = None
    # 订单佣金
    self.buycomm = None

    # 定义变量保存所有收盘价
    self.dataclose = self.data.close

    #计算10日均线
    self.sma10 = btind.MovingAverageSimple(self.dataclose, period=self.params.period_sma5)
    # 计算30日均线
    self.sma30 = btind.MovingAverageSimple(self.dataclose, period=self.params.period_sma20)


4.当执行cerebro.run()时，会循环执行next()方法，在方法中会执行策略逻辑，分别完成买（self.order=self.buy()）、卖（self.order=self.sell()）操作；当执行这两个操作时，又会调用订单变动通知方法（notify_order()）及交易通知方法（notify_trade()），如此反复，直至数据循环完毕。

5.notify_order()方法：一个订单从提交到被cerebro接受至少触发三个通知，分别为：Sumbitted、Accepted、Completed。代码中，当触发前两个通知时，什么事都不做，当触发Completed时，将调用log()方法打印出订单信息。而后续的Canceled、Margin、Rejected通知只有在订单没有完成（有可能是因为保证金不足…）时才触发。详情参考：Order Status values.

6.notify_trade()方法：在order状态执行完毕后，会进入notify_trade中执行log()方法打印出盈利及手续费 。

In [ ]:
def notify_order(self, order):

    # 等待订单提交、订单被cerebro接受
    if order.status in [order.Submitted, order.Accepted]:
        return

    # 等待订单完成
    if order.status in [order.Completed]:
        if order.isbuy():
            self.log(
                'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                (order.executed.price,
                 order.executed.value,
                 order.executed.comm))

        else:
            self.log(
                'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                (order.executed.price,
                 order.executed.value,
                 order.executed.comm))

    # 如果订单保证金不足，将不会完成，而是执行以下拒绝程序
    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        self.log('Order Canceled/Margin/Rejected')

    self.order = None

def notify_trade(self, trade):

    if not trade.isclosed:
        return

    self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
             (trade.pnl, trade.pnlcomm))  # pnl：盈利  pnlcomm：手续费
